# Streaming Consumer for Measuring Latency

In [3]:
%matplotlib inline
import os, sys
import commands
import radical.pilot as rp
import random
import pandas as pd
import ast
import seaborn as sns

from pykafka import KafkaClient
import numpy as np
import time
import datetime
import dateutil.parser
import ast
import sklearn.cluster
import pickle

In [4]:
run_timestamp=datetime.datetime.now()
RESULT_FILE= "results/kafka-throughput-consumer-pilot-" + run_timestamp.strftime("%Y%m%d-%H%M%S") + ".csv"
try:
    os.makedirs("results")
except:
    pass
output_file=open(RESULT_FILE, "w")

# Benchmark

Streaming Data from Kafka

    mongod --dbpath=/gpfs/flash/users/tg804093/mongo
    
see `rp_kmeans_streaming.py` for worker script.

## Setup Pilot

In [5]:
def print_details(detail_object):
    if type(detail_object)==str:
        detail_object = ast.literal_eval(detail_object)
    for i in detail_object:
        detail_object[i]=str(detail_object[i])
    return pd.DataFrame(detail_object.values(), 
             index=detail_object.keys(), 
             columns=["Value"])

os.environ["RADICAL_PILOT_VERBOSE"]="ERROR"
os.environ["RADICAL_SAGA_PTY_VERBOSE"]="ERROR" 
#os.environ["RADICAL_PILOT_DBURL"]="mongodb://mongo.radical-cybertools.org:24242/sc15-test000"
os.environ["RADICAL_PILOT_DBURL"]="mongodb://c251-120:27017/sc15-test000"
session = rp.Session()
pmgr = rp.PilotManager(session=session)
umgr = rp.UnitManager (session=session,
                       scheduler=rp.SCHED_ROUND_ROBIN)
print "Session id: %s Pilot Manager: %s" % (session.uid, str(pmgr.as_dict()))

pdesc = rp.ComputePilotDescription()
pdesc.resource = "local.localhost_anaconda"  # NOTE: This is a "label", not a hostname
pdesc.runtime  = 120 # minutes
pdesc.cores    = 24
pdesc.cleanup  = False
pilot = pmgr.submit_pilots(pdesc)
umgr.add_pilots(pilot)

Session id: rp.session.c251-120.wrangler.tacc.utexas.edu.tg804093.017307.0008 Pilot Manager: {'uid': 'pmgr.0000'}


## Run Benchmarks

In [7]:
# configuration
number_centroids = 10
number_dimensions = 3
number_messages = 1000
number_cus = 1
repeats = 3

r = redis.StrictRedis(host=redis_host, port=6379, db=0)

def put_model(model):
    r.set('kmeans', pickle.dumps(model))
    
def get_model():
    return pickle.loads(r.get("kmeans"))

centroids = np.random.randn(number_centroids, number_dimensions)
kmeans = sklearn.cluster.MiniBatchKMeans(n_clusters=len(centroids), init=centroids, n_init=1)
   

per_cu_messages = number_messages/number_cus

cudesc_list=[]
for i in range(number_cus):
    cudesc = rp.ComputeUnitDescription()
    cudesc.executable  = "python"
    cudesc.arguments   = [os.path.join(os.getcwd(), 'rp_kmeans_streaming.py'), 100, 1, 1]
    cudesc.cores       = 1

    cudesc_list.append(cudesc)
    
    

2017-05-21 18:31:04,607: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : PTYProcess init <saga.utils.pty_process.PTYProcess object at 0x2b4d15ebd050>
2017-05-21 18:31:04,607: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: INFO    : running: /usr/bin/env TERM=vt100 /bin/bash -i
2017-05-21 18:31:04,610: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : write: [  141] [   33] ( export PS1='$' ; set prompt='$'\n)
2017-05-21 18:31:04,828: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : read : [  141] [  159] (]0;c251-120.wrangler.tacc.utexas.educ251-120.wrangler ~/notebooks/Pilot-Memory/examples/jupyter/streaming/throughput$ ]0;c251-120.wrangler.tacc.utexas.edu$)
2017-05-21 18:31:04,828: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : got initial shell prompt (6) (]0;c251-120.wrangler.tacc.utexa

In [8]:
print "Submit Compute Units to Unit Manager ..."
cu_set = umgr.submit_units(cudesc_list)
print "Waiting for CUs to complete ..."
umgr.wait_units()
print "All CUs completed successfully!"
#cu_results = cu_set[0]
details=cu_results.as_dict()

Submit Compute Units to Unit Manager ...
Waiting for CUs to complete ...


2017-05-21 18:31:55,912: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : PTYProcess init <saga.utils.pty_process.PTYProcess object at 0x2b4d17050250>
2017-05-21 18:31:55,913: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: INFO    : running: /usr/bin/env TERM=vt100 /bin/bash -i
2017-05-21 18:31:55,916: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : write: [  143] [   33] ( export PS1='$' ; set prompt='$'\n)
2017-05-21 18:31:56,130: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : read : [  143] [   38] (]0;c251-120.wrangler.tacc.utexas.edu)
2017-05-21 18:31:56,131: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : read : [  143] [   82] (c251-120.wrangler ~/notebooks/Pilot-Memory/examples/jupyter/streaming/throughput$ )
2017-05-21 18:31:56,131: radical.saga.pty    : MainProcess                 

2017-05-21 18:31:57,190: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : write: [  145] [   83] (/bin/sh $HOME/.saga/adaptors/shell_job//wrapper.sh $HOME/.saga/adaptors/shell_job/\n)
2017-05-21 18:31:57,196: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : read : [  145] [   12] (PID: 12250\n)
2017-05-21 18:31:57,737: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : read : [  145] [   12] (PROMPT-0->\n)
2017-05-21 18:31:57,738: radical.saga.pty    : MainProcess                     : Thread-12      : DEBUG   : flush: [  145] [     ] (flush pty read cache)
2017-05-21 18:31:57,738: radical.saga.pty    : MainProcess                     : Thread-12      : WARNING : flush: [  145] [    1] (discard data: '
')
2017-05-21 18:31:57,738: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : flush: [  143] [     ] (flush pty read cache)
2017-05

All CUs completed successfully!


2017-05-21 18:32:56,163: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : flush: [  143] [     ] (flush pty read cache)
2017-05-21 18:32:56,163: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: WARNING : flush: [  143] [    1] (discard data: '
')
2017-05-21 18:32:56,264: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : write: [  143] [    5] (LIST\n)
2017-05-21 18:32:56,268: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : read : [  143] [   13] (OK\n11277.0\n)
2017-05-21 18:32:56,269: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : read : [  143] [   12] (PROMPT-0->\n)
2017-05-21 18:32:56,272: radical.saga.pty    : MainProcess                     : PilotLauncherWorker-1: DEBUG   : flush: [  143] [     ] (flush pty read cache)
2017-05-21 18:32:56,272: radical.saga.pty    : MainProcess              

In [2]:
session.close()
del session

NameError: name 'session' is not defined